In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import math

In [2]:
df = pd.read_csv("historical_data.csv")

In [46]:
df

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,actual_duration
0,1.0,2015-02-06 22:24:17,2015-02-06 23:27:16,1845,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0,3779.0
1,2.0,2015-02-10 21:49:25,2015-02-10 22:56:29,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0,4024.0
2,3.0,2015-01-22 20:39:28,2015-01-22 21:09:09,5477,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0,446,690.0,1781.0
3,3.0,2015-02-03 21:21:45,2015-02-03 22:13:00,5477,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0,446,289.0,3075.0
4,3.0,2015-02-15 02:40:36,2015-02-15 03:20:26,5477,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0,446,650.0,2390.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197423,1.0,2015-02-17 00:19:41,2015-02-17 01:24:48,2956,fast,4.0,3,1389,3,345,649,17.0,17.0,23.0,251,331.0,3907.0
197424,1.0,2015-02-13 00:01:59,2015-02-13 00:58:22,2956,fast,4.0,6,3010,4,405,825,12.0,11.0,14.0,251,915.0,3383.0
197425,1.0,2015-01-24 04:46:08,2015-01-24 05:36:16,2956,fast,4.0,5,1836,3,300,399,39.0,41.0,40.0,251,795.0,3008.0
197426,1.0,2015-02-01 18:18:15,2015-02-01 19:23:22,3630,sandwich,1.0,1,1175,1,535,535,7.0,7.0,12.0,446,384.0,3907.0


# What to do :
- Get total duration by subtracting "actual_delivery_time" by "created_at"
- do something about NaN
- feature engineering
- choose features 
- choose model

In [20]:
df.dtypes

market_id                                       float64
created_at                                       object
actual_delivery_time                             object
store_id                                          int64
store_primary_category                           object
order_protocol                                  float64
total_items                                       int64
subtotal                                          int64
num_distinct_items                                int64
min_item_price                                    int64
max_item_price                                    int64
total_onshift_dashers                           float64
total_busy_dashers                              float64
total_outstanding_orders                        float64
estimated_order_place_duration                    int64
estimated_store_to_consumer_driving_duration    float64
actual_duration                                 float64
dtype: object

In [15]:
df['actual_duration'] = (pd.to_datetime(df['actual_delivery_time']) - pd.to_datetime(df['created_at'])).dt.total_seconds()

In [31]:
print(f"Average time of delivery (Seconds): {round(df['actual_duration'].mean())}")
print(f"Average time of delivery (Minutes): {round(df['actual_duration'].mean())/60}")
print(f"Average time of delivery (Hours): {round(df['actual_duration'].mean())/3600}")

Average time of delivery (Seconds): 2908
Average time of delivery (Minutes): 48.46666666666667
Average time of delivery (Hours): 0.8077777777777778


In [47]:
# check which columns have NaN values and how many
df.isna().sum()

market_id                                         987
created_at                                          0
actual_delivery_time                                7
store_id                                            0
store_primary_category                           4760
order_protocol                                    995
total_items                                         0
subtotal                                            0
num_distinct_items                                  0
min_item_price                                      0
max_item_price                                      0
total_onshift_dashers                           16262
total_busy_dashers                              16262
total_outstanding_orders                        16262
estimated_order_place_duration                      0
estimated_store_to_consumer_driving_duration      526
actual_duration                                     0
dtype: int64

In [44]:
# average difference of values difference from the mean 
(df["actual_duration"]-df['actual_duration'].mean()).mean()
# very low so i will replace all NaN values with mean

2.9402945910493497e-13

In [45]:
df['actual_duration'].fillna(df['actual_duration'].mean(), inplace=True)

In [48]:
16262/197428 

0.08236926879672589

### there are 16262 Nan values for total_onshift_dashers, total_busy_dashers, total_outstanding_order which account for 8 percent of so ill just drop those rows

In [51]:
x = df.dropna(subset=['total_onshift_dashers']).reset_index()

In [53]:
x.isna().sum()

index                                              0
market_id                                        919
created_at                                         0
actual_delivery_time                               7
store_id                                           0
store_primary_category                          4215
order_protocol                                   917
total_items                                        0
subtotal                                           0
num_distinct_items                                 0
min_item_price                                     0
max_item_price                                     0
total_onshift_dashers                              0
total_busy_dashers                                 0
total_outstanding_orders                           0
estimated_order_place_duration                     0
estimated_store_to_consumer_driving_duration     482
actual_duration                                    0
dtype: int64

In [55]:
df.dropna(subset=['total_onshift_dashers'], inplace=True)
df = df.reset_index()

In [60]:
df.dropna(subset=['estimated_store_to_consumer_driving_duration',"order_protocol", "store_primary_category", "market_id" ], inplace=True)
df = df.reset_index()

In [66]:
df

,level_0,index,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,actual_duration
0,0,0,1.0,2015-02-06 22:24:17,2015-02-06 23:27:16,1845,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0,3779.0
1,1,1,2.0,2015-02-10 21:49:25,2015-02-10 22:56:29,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0,4024.0
2,8,8,2.0,2015-02-16 00:11:35,2015-02-16 00:38:01,5477,indian,3.0,4,4771,3,820,1604,8.0,6.0,18.0,446,289.0,1586.0
3,14,14,1.0,2015-02-12 03:36:46,2015-02-12 04:14:39,2841,italian,1.0,1,1525,1,1525,1525,5.0,6.0,8.0,446,795.0,2273.0
4,15,15,1.0,2015-01-27 02:12:36,2015-01-27 03:02:24,2841,italian,1.0,2,3620,2,1425,2195,5.0,5.0,7.0,446,205.0,2988.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175779,181161,197423,1.0,2015-02-17 00:19:41,2015-02-17 01:24:48,2956,fast,4.0,3,1389,3,345,649,17.0,17.0,23.0,251,331.0,3907.0
175780,181162,197424,1.0,2015-02-13 00:01:59,2015-02-13 00:58:22,2956,fast,4.0,6,3010,4,405,825,12.0,11.0,14.0,251,915.0,3383.0
175781,181163,197425,1.0,2015-01-24 04:46:08,2015-01-24 05:36:16,2956,fast,4.0,5,1836,3,300,399,39.0,41.0,40.0,251,795.0,3008.0
175782,181164,197426,1.0,2015-02-01 18:18:15,2015-02-01 19:23:22,3630,sandwich,1.0,1,1175,1,535,535,7.0,7.0,12.0,446,384.0,3907.0


In [67]:
df.store_primary_category.unique()

array(['american', 'mexican', 'indian', 'italian', 'sandwich', 'thai',
       'cafe', 'salad', 'pizza', 'chinese', 'singaporean', 'burger',
       'mediterranean', 'japanese', 'greek', 'catering', 'filipino',
       'convenience-store', 'other', 'vegan', 'asian', 'barbecue',
       'breakfast', 'fast', 'dessert', 'smoothie', 'seafood',
       'vietnamese', 'cajun', 'steak', 'middle-eastern', 'persian',
       'nepalese', 'korean', 'sushi', 'latin-american', 'chocolate',
       'burmese', 'hawaiian', 'british', 'pasta', 'alcohol', 'vegetarian',
       'dim-sum', 'peruvian', 'turkish', 'ethiopian', 'bubble-tea',
       'german', 'french', 'caribbean', 'gluten-free', 'comfort-food',
       'gastropub', 'afghan', 'pakistani', 'moroccan', 'tapas',
       'malaysian', 'soup', 'brazilian', 'european', 'cheese', 'african',
       'argentine', 'kosher', 'irish', 'spanish', 'russian', 'southern',
       'lebanese', 'belgian', 'alcohol-plus-food'], dtype=object)

In [65]:
df.isna().sum()

level_0                                         0
index                                           0
market_id                                       0
created_at                                      0
actual_delivery_time                            0
store_id                                        0
store_primary_category                          0
order_protocol                                  0
total_items                                     0
subtotal                                        0
num_distinct_items                              0
min_item_price                                  0
max_item_price                                  0
total_onshift_dashers                           0
total_busy_dashers                              0
total_outstanding_orders                        0
estimated_order_place_duration                  0
estimated_store_to_consumer_driving_duration    0
actual_duration                                 0
dtype: int64

In [69]:
df.drop(columns="level_0", inplace = True)

In [72]:
df.drop(columns="index", inplace = True)

In [74]:
df.head()

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,actual_duration
0,1.0,2015-02-06 22:24:17,2015-02-06 23:27:16,1845,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0,3779.0
1,2.0,2015-02-10 21:49:25,2015-02-10 22:56:29,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0,4024.0
2,2.0,2015-02-16 00:11:35,2015-02-16 00:38:01,5477,indian,3.0,4,4771,3,820,1604,8.0,6.0,18.0,446,289.0,1586.0
3,1.0,2015-02-12 03:36:46,2015-02-12 04:14:39,2841,italian,1.0,1,1525,1,1525,1525,5.0,6.0,8.0,446,795.0,2273.0
4,1.0,2015-01-27 02:12:36,2015-01-27 03:02:24,2841,italian,1.0,2,3620,2,1425,2195,5.0,5.0,7.0,446,205.0,2988.0


### features to add
- cost per item => subtotal/total_items
- cost per distinct item => subtotal/num_distinct_items

In [75]:
df["cost_per_item"] = df["subtotal"]/df["total_items"]
df["cost_per_distinct_item"] = df["subtotal"]/df["num_distinct_items"]

In [78]:
df.drop(columns= "total_fraction_of_available_dashers", inplace= True)

In [100]:
df.head()

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,actual_duration,cost_per_item,cost_per_distinct_item
0,1.0,2015-02-06 22:24:17,2015-02-06 23:27:16,1845,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0,3779.0,860.25,860.250000
1,2.0,2015-02-10 21:49:25,2015-02-10 22:56:29,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0,4024.0,1900.00,1900.000000
2,2.0,2015-02-16 00:11:35,2015-02-16 00:38:01,5477,indian,3.0,4,4771,3,820,1604,8.0,6.0,18.0,446,289.0,1586.0,1192.75,1590.333333
3,1.0,2015-02-12 03:36:46,2015-02-12 04:14:39,2841,italian,1.0,1,1525,1,1525,1525,5.0,6.0,8.0,446,795.0,2273.0,1525.00,1525.000000
4,1.0,2015-01-27 02:12:36,2015-01-27 03:02:24,2841,italian,1.0,2,3620,2,1425,2195,5.0,5.0,7.0,446,205.0,2988.0,1810.00,1810.000000


## we need to predict a time which is a regression problem

In [96]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [101]:
X = df[[ "total_items", "subtotal",
    "num_distinct_items", "min_item_price", "max_item_price",
    "total_onshift_dashers", "total_busy_dashers", "total_outstanding_orders",
    "estimated_order_place_duration",
    "estimated_store_to_consumer_driving_duration", "actual_duration",
    "cost_per_item", "cost_per_distinct_item"
]]

In [102]:
y = df["actual_duration"]

In [103]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [104]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(131832, 13)
(131832,)
(43945, 13)
(43945,)


In [111]:
linreg = LinearRegression()
linreg.fit(X, y)

LinearRegression()

In [112]:
y_pred = linreg.predict(X_test)
#print(metrics.accuracy_score(y_test, y_pred))

In [113]:
y_test

50611     2905.0
49183     2971.0
4527       963.0
132       2824.0
57098     2372.0
           ...  
173657    4655.0
116886    4033.0
150628    1789.0
32051     2774.0
17136     2507.0
Name: actual_duration, Length: 43945, dtype: float64

In [114]:
y_pred

array([2905., 2971.,  963., ..., 1789., 2774., 2507.])

In [116]:
[x1 - x2 for (x1, x2) in zip(y_pred, y_test)]

[1.3642420526593924e-12,
 8.185452315956354e-12,
 1.2050804798491299e-11,
 -4.547473508864641e-13,
 2.2737367544323206e-12,
 -6.366462912410498e-12,
 3.410605131648481e-12,
 4.092726157978177e-12,
 3.183231456205249e-12,
 -3.183231456205249e-12,
 5.4569682106375694e-12,
 2.7284841053187847e-12,
 5.4569682106375694e-12,
 6.366462912410498e-12,
 2.5011104298755527e-12,
 -4.547473508864641e-13,
 4.547473508864641e-13,
 -6.366462912410498e-12,
 -2.000888343900442e-11,
 6.821210263296962e-12,
 3.183231456205249e-12,
 -1.8189894035458565e-12,
 -1.318767317570746e-11,
 1.3642420526593924e-12,
 5.6843418860808015e-12,
 8.412825991399586e-12,
 3.637978807091713e-12,
 4.547473508864641e-12,
 1.3642420526593924e-12,
 1.5916157281026244e-12,
 2.5011104298755527e-12,
 4.774847184307873e-12,
 3.183231456205249e-12,
 4.547473508864641e-13,
 -3.183231456205249e-12,
 0.0,
 1.0913936421275139e-11,
 -2.2737367544323206e-12,
 9.094947017729282e-13,
 -9.094947017729282e-13,
 1.1368683772161603e-11,
 5.2295

In [ ]:
linreg.predict()